In [1]:
import pandas as pd

# P4C2T4T5

In [27]:
df = pd.read_csv('/Volumes/sd/faith/MTCSB/projects/P4-barcoding_strains/20241029_analysis/rerun/P4C2T4T5/FP/ST1/TableOfEstimates.csv')
df['project'] = df['Unnamed: 0'].apply(lambda x: str(x).split("_")[0])
df['sample_id'] = df['Unnamed: 0'].apply(lambda x: str(x).split("_")[1].split(".")[0])
df
#P4C2T4:
df_P4C2T4 = pd.read_csv('/Users/ruprec01/Documents/Faith_lab/Git/population_bottlenecks/input_files/metadata/P4C2T4_cr_mouse_sample - P4C2T4.tsv', sep = '\t')

df_P4C2T5 = pd.read_csv('/Users/ruprec01/Documents/Faith_lab/Git/population_bottlenecks/input_files/metadata/P4C2T5_cr_mouse_sample - P4C2T5.tsv', sep = '\t')

df_meta = pd.concat([df_P4C2T4, df_P4C2T5], axis = 0)
df_meta

df_merge = pd.merge(df, df_meta, on = ['sample_id', 'project'], how = 'left')
df_merge.columns

Index(['Unnamed: 0', 'TotalReads', 'Number of barcodes', 'Ns_MinCutoff', 'Nb',
       'Ns', 'AverageFrequency', 'CFU', 'Log10Ns', 'Log10CFU', 'CFU/Ns',
       'project', 'sample_id', 'sample_type', 'date_collected', 'time_point_h',
       'cage_id', 'mouse_type', 'pretreatment', 'mouse_id', 'gender',
       'gavage_dose_200ul', 'strain_bcs', 'strains', 'tube_type',
       'tube_weight_empty', 'tube_weight_full', 'stool_weight_mg',
       'cfu_g_stool', 'cfu_incubation_time', 'dna_conc_ng_ul',
       'dna_conc_ng_ul.1'],
      dtype='object')

gd_name                           sample_id  project
P4C2T4 SPF NA                     s27        P4C2T4     P4C2T4_s27.F2_stool.DNA
                                  s29        P4C2T4     P4C2T4_s29.G2_stool.DNA
P4C2T4 SPF antibiotic_vancomycin  s25        P4C2T4     P4C2T4_s25.E2_stool.DNA
P4C2T4 germ_free NA               s17        P4C2T4     P4C2T4_s17.H1_stool.DNA
                                  s19        P4C2T4     P4C2T4_s19.B2_stool.DNA
P4C2T4 germ_free donor_BSD2780    s21        P4C2T4     P4C2T4_s21.C2_stool.DNA
                                  s23        P4C2T4     P4C2T4_s23.D2_stool.DNA
P4C2T5 SPF NA                     s43        P4C2T5     P4C2T5_s43.G3_stool.DNA
                                  s45        P4C2T5     P4C2T5_s45.H3_stool.DNA
                                  s47        P4C2T5     P4C2T5_s47.A4_stool.DNA
P4C2T5 SPF antibiotic_vancomycin  s37        P4C2T5     P4C2T5_s37.C3_stool.DNA
                                  s39        P4C2T5     P4C2T5_s39.

In [45]:
df_select = df_merge.loc[(df_merge['sample_type'] == 'stool DNA') & (df_merge['time_point_h'] == '24 h')].copy()
df_select['gd_name'] = df_select['project'] + " " +df_select['mouse_type'] + " " + df_select['pretreatment'].fillna('NA')

#gets first of duplicate sequencing
df_select.sort_values(['mouse_type', 'pretreatment','cage_id'], ascending = True, inplace = True)

df_select = df_select.groupby(['gd_name', 'sample_id' ])['Unnamed: 0'].first()
df_select = df_select.reset_index()

#create Sample,Group,Order,BioSample csv
df_gd_meta = df_select[['Unnamed: 0', 'gd_name']].copy()
df_gd_meta['Group'] = 1
df_gd_meta = df_gd_meta.rename(columns = {'Unnamed: 0': 'Sample', 'gd_name': 'BioSample'})
df_gd_meta.reset_index(drop = True, inplace = True)
df_gd_meta['Order'] = df_gd_meta.index+1

df_gd_meta[["Sample","Group","Order","BioSample"]].to_csv('/Users/ruprec01/Documents/Faith_lab/Git/population_bottlenecks/input_files/gd_sheets/P4C2T4T5/24h_stool_gdmeta.csv',
                                                            index = False)